In [18]:
import pandas as pd
import numpy as np
import re
from functools import reduce
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error

In [19]:
# --- 1. Read and merge MASTER.xlsx sheets ---

all_sheets = pd.read_excel("MASTER.xlsx", sheet_name=None)
dfs = []
gp_df = None

for name, df in all_sheets.items():
    df = df.rename(columns=lambda c: c.upper().strip())
    if "PLAYER" not in df.columns or "GP" not in df.columns:
        raise KeyError(f"Sheet '{name}' missing PLAYER or GP column.")
    if name.strip().upper() == "GENERAL - DEFENSE":
        gp_df = df[["PLAYER", "GP"]].drop_duplicates()
    dfs.append(df.drop(columns="GP"))

if gp_df is None:
    raise KeyError("Could not find sheet 'General - Defense' to extract GP.")

merged_df = reduce(lambda L, R: pd.merge(L, R, on="PLAYER", how="outer"), dfs)
merged_df = merged_df.merge(gp_df, on="PLAYER", how="left")

In [20]:
# --- 2. Merge Def Impact Score from Rankings.xlsx ---

scores = pd.read_excel("Rankings.xlsx", sheet_name="Def Impact Score")
scores = scores.rename(columns=lambda c: c.upper().strip())[["PLAYER", "SCORE"]]

def clean_name(name: str) -> str:
    name = name.upper().strip()
    name = re.sub(r"[.\-]", "", name)
    return re.sub(r"[^A-Z0-9 ]", ".", name)

merged_df["PLAYER"] = merged_df["PLAYER"]
scores["PLAYER"]    = scores["PLAYER"]

merged_df = (
    merged_df.merge(scores, on="PLAYER", how="left")
             .dropna(subset=["SCORE"])
)

merged_df = merged_df.sort_values(by="SCORE", ascending=False).reset_index(drop=True)

# Print unmatched players
unmatched_players = set(scores["PLAYER"]) - set(merged_df["PLAYER"])
print("Players in Rankings.xlsx not matched in merged_df:")
print(sorted(unmatched_players))

merged_df

Players in Rankings.xlsx not matched in merged_df:
['CJ\xa0McCollum', 'Dorian Finney‑Smith', 'Jaren Jackson\xa0Jr.', 'Kelly Oubre\xa0Jr.', 'Kentavious Caldwell‑Pope', 'PJ Tucker', 'Robert Williams\xa0III']


,PLAYER,DREB,DREB%,%DREB,STL,STL%,BLK,%BLK,OPP PTS OFF TOV,OPP PTS 2ND CHANCE,...,CONTESTED SHOTS,BOX OUTS,DEF BOX OUTS,TEAM REB ON BOX OUTS,PLAYER REB ON BOX OUTS,% BOX OUTS DEF,% TEAM REB WHEN BOX OUT,% PLAYER REB WHEN BOX OUT,GP,SCORE
0,Rudy Gobert,9.2,25.6,36.8,0.7,13.2,2.1,50.0,12.1,8.7,...,11.0,2.8,1.9,2.8,1.9,67.0,99.5,68.4,76,561.0
1,Victor Wembanyama,8.4,27.3,38.9,1.2,27.4,3.6,70.8,11.7,8.0,...,9.1,0.9,0.6,0.8,0.5,67.2,100.0,57.6,71,560.0
2,Bam Adebayo,8.1,24.2,34.9,1.1,20.0,0.9,37.3,10.5,9.5,...,6.7,3.1,2.8,2.9,1.4,90.0,97.6,48.8,71,559.0
3,Herb Jones,2.6,8.4,12.1,1.4,26.3,0.8,25.3,9.5,7.7,...,6.1,0.8,0.8,0.8,0.3,98.4,95.2,30.6,76,558.0
4,Anthony Davis,9.5,25.2,36.6,1.2,21.9,2.3,52.5,12.0,10.6,...,11.2,2.5,1.8,2.5,1.7,74.1,98.4,68.8,76,557.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Javonte Green,5.2,21.9,29.9,1.1,24.4,0.9,40.0,7.9,6.0,...,3.1,1.6,1.1,1.6,0.9,71.4,100.0,57.1,9,473.0
81,Torrey Craig,2.8,14.8,21.6,0.6,16.9,0.4,22.6,5.7,5.5,...,4.0,0.4,0.4,0.4,0.3,86.4,100.0,63.6,53,471.0
82,Hamidou Diallo,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,470.0
83,Theo Maledon,1.2,9.3,14.1,0.4,23.3,0.0,0.0,6.4,3.0,...,2.0,0.1,0.1,0.1,0.0,100.0,100.0,0.0,17,469.0


In [21]:
# --- 3. Prepare features and target (model training) ---

numeric_cols = merged_df.select_dtypes(include="number").columns.drop("SCORE")
X = merged_df[numeric_cols].values
y = merged_df["SCORE"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [22]:
# --- 4. Fit RandomForest with LOOCV evaluation ---

rf = RandomForestRegressor(n_estimators=100, random_state=42)

loo = LeaveOneOut()
y_pred = cross_val_predict(rf, X_scaled, y, cv=loo)

r2 = r2_score(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)
print(f"RandomForest LOOCV R²:  {r2:.3f}")
print(f"RandomForest LOOCV RMSE: {rmse:.3f}")

# Fit final model on full data
rf.fit(X_scaled, y)
importances = pd.Series(rf.feature_importances_, index=numeric_cols)
importances = importances.sort_values(ascending=False)

print("\nRandomForest Feature Importances:")
print(importances)

RandomForest LOOCV R²:  0.307
RandomForest LOOCV RMSE: 22.877

RandomForest Feature Importances:
DEFLECTIONS                    0.104878
CONTESTED 3 PT SHOTS           0.077597
OPP PTS OFF TOV                0.068491
% TEAM REB WHEN BOX OUT        0.057443
DFGA                           0.055807
OPP PTS 2ND CHANCE             0.052327
DIFF%                          0.049498
CONTESTED SHOTS                0.046730
DFG%                           0.042888
BLK                            0.039459
DFGM                           0.036987
GP                             0.035382
STL%                           0.032402
OPP PTS FB                     0.030657
%BLK                           0.029867
OPP PTS PAINT                  0.024795
STL                            0.023232
% BOX OUTS DEF                 0.021973
% PLAYER REB WHEN BOX OUT      0.021237
FG%                            0.020633
DREB%                          0.018762
CONTESTED 2PT SHOTS            0.014642
%DREB                  

c:\Users\nadee_4wl0sna\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [23]:
# --- 5. Add new impact score based on dynamic feature importances ---

# Work on a new copy of merged_df to avoid scaled columns
output_df = merged_df.copy()

# Apply weights only to columns that exist
for feature, weight in importances.items():
    if feature in output_df.columns:
        output_df[feature + "_WEIGHTED"] = output_df[feature] * weight

# Compute impact score
weighted_cols = [col for col in output_df.columns if col.endswith("_WEIGHTED")]
output_df["IMPACT SCORE"] = output_df[weighted_cols].sum(axis=1)

In [24]:
# --- 6. Filter by GP after metric creation ---
output_df = output_df[output_df["GP"] >= 65]

In [25]:
# --- 7. Sort and save to CSV (with original numeric columns unscaled) ---
output_df = output_df.sort_values("IMPACT SCORE", ascending=False)
output_df.to_csv("OUTPUT.csv", index=False)